In [25]:
from tbparse import SummaryReader
import polars as pl

In [26]:
logs = SummaryReader("runs/", pivot=True, extra_columns={'dir_name'})
params = pl.from_pandas(logs.hparams, nan_to_null=True)
scores = pl.from_pandas(logs.scalars, nan_to_null=True)
hparams = params.join(scores.select(pl.col("F-Score", "dir_name")), on='dir_name', how='inner')

In [64]:
def get_model_settings(model_type: str) -> (pl.DataFrame, pl.DataFrame):
    model = hparams.filter(pl.col("model_settings").str.contains(model_type))
    non_null = model.select(pl.all().is_not_null().all()).row(0)
    model = model[:, non_null]
    different = model.select(pl.all().n_unique() > 1).row(0)
    diff = model[:, different].sort("F-Score", descending=True)
    iden = model.select(pl.all().n_unique() == 1).row(0)
    identical = model[:, iden].select(pl.all().exclude("dir_name", "F-Score")).limit(1)
    
    return diff, identical

In [66]:
unique, identical = get_model_settings("mamba")
print(identical)
unique

shape: (1, 27)
┌────────────┬───────┬────────┬────────┬───┬───────────┬───────────────┬──────────────┬────────────┐
│ activation ┆ beats ┆ causal ┆ center ┆ … ┆ scheduler ┆ splits        ┆ test_batch_s ┆ train_set  │
│ ---        ┆ ---   ┆ ---    ┆ ---    ┆   ┆ ---       ┆ ---           ┆ ize          ┆ ---        │
│ str        ┆ f64   ┆ f64    ┆ f64    ┆   ┆ f64       ┆ str           ┆ ---          ┆ str        │
│            ┆       ┆        ┆        ┆   ┆           ┆               ┆ f64          ┆            │
╞════════════╪═══════╪════════╪════════╪═══╪═══════════╪═══════════════╪══════════════╪════════════╡
│ SELU       ┆ 0.0   ┆ 1.0    ┆ 1.0    ┆ … ┆ 1.0       ┆ (0.85, 0.15,  ┆ 1.0          ┆ a2md_train │
│            ┆       ┆        ┆        ┆   ┆           ┆ 0.0)          ┆              ┆            │
└────────────┴───────┴────────┴────────┴───┴───────────┴───────────────┴──────────────┴────────────┘


batch_size,dropout,epochs,flux,learning_rate,mapping,min_test_score,model_settings,n_layers,num_channels,onset_cooldown,pr_points,time_shift,weight_decay,dir_name,F-Score
f64,f64,f64,f64,f64,str,f64,str,f64,f64,f64,f64,f64,f64,str,f64
4.0,0.1,20.0,0.0,0.0001,"""THREE_CLASS_STANDARD""",0.54,"""mamba""",3.0,16.0,0.021,1000.0,0.015,0.00001,"""Jun10_17-58-15_seppel-liemarce…",0.455403
16.0,0.1,20.0,1.0,0.0001,"""Three class standard""",0.48,"""mamba_fast""",5.0,32.0,0.021,1000.0,0.02,0.00001,"""Jun19_13-16-07_seppel-liemarce…",0.434372
16.0,0.1,20.0,0.0,0.0001,"""Three class standard""",0.48,"""mamba_fast""",3.0,16.0,0.021,1000.0,0.02,0.00001,"""Jun25_07-53-16_seppel-liemarce…",0.139301
24.0,0.1,20.0,1.0,0.0001,"""Three class standard""",0.48,"""mamba_fast""",5.0,16.0,0.021,200.0,0.025,0.00001,"""Jun23_17-34-02_seppel-liemarce…",0.116948
24.0,0.1,20.0,1.0,0.0001,"""Three class standard""",0.48,"""mamba_fast""",10.0,16.0,0.021,200.0,0.025,0.00001,"""Jun22_16-25-42_seppel-liemarce…",0.052223
32.0,0.3,30.0,0.0,0.00005,"""THREE_CLASS_STANDARD""",0.54,"""mamba""",3.0,16.0,0.02,1000.0,0.015,0.0,"""Jun08_13-22-15_seppel-liemarce…",0.04605


In [67]:
unique, identical = get_model_settings("attention")
print(identical)
unique

shape: (1, 26)
┌────────────┬───────┬────────┬────────┬───┬─────────────┬───────────┬────────────────┬────────────┐
│ activation ┆ beats ┆ causal ┆ center ┆ … ┆ sample_rate ┆ scheduler ┆ splits         ┆ train_set  │
│ ---        ┆ ---   ┆ ---    ┆ ---    ┆   ┆ ---         ┆ ---       ┆ ---            ┆ ---        │
│ str        ┆ f64   ┆ f64    ┆ f64    ┆   ┆ f64         ┆ f64       ┆ str            ┆ str        │
╞════════════╪═══════╪════════╪════════╪═══╪═════════════╪═══════════╪════════════════╪════════════╡
│ SELU       ┆ 0.0   ┆ 1.0    ┆ 1.0    ┆ … ┆ 44100.0     ┆ 1.0       ┆ (0.85, 0.15,   ┆ a2md_train │
│            ┆       ┆        ┆        ┆   ┆             ┆           ┆ 0.0)           ┆            │
└────────────┴───────┴────────┴────────┴───┴─────────────┴───────────┴────────────────┴────────────┘


batch_size,context_size,dataset_version,epochs,flux,learning_rate,mapping,min_test_score,num_attention_blocks,num_channels,num_heads,onset_cooldown,pad_annotations,test_batch_size,time_shift,dir_name,F-Score
f64,f64,str,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,str,f64
16.0,200.0,"""L""",20.0,1.0,0.0001,"""THREE_CLASS_STANDARD""",0.54,5.0,24.0,8.0,0.021,1.0,1.0,0.015,"""Jun13_10-57-29_seppel-liemarce…",0.492189
16.0,200.0,"""L""",20.0,1.0,0.0001,"""THREE_CLASS_STANDARD""",0.54,5.0,32.0,8.0,0.021,1.0,1.0,0.015,"""Jun14_09-33-24_seppel-liemarce…",0.482672
4.0,200.0,"""L""",20.0,0.0,0.0001,"""THREE_CLASS_STANDARD""",0.54,5.0,16.0,8.0,0.021,1.0,1.0,0.015,"""Jun11_13-54-21_seppel-liemarce…",0.458652
16.0,200.0,"""L""",20.0,1.0,0.0001,"""Three class standard""",0.48,5.0,24.0,8.0,0.021,1.0,1.0,0.02,"""Jun17_15-04-11_seppel-liemarce…",0.393078
4.0,200.0,"""S""",20.0,0.0,0.0001,"""THREE_CLASS_STANDARD""",0.54,5.0,32.0,8.0,0.021,1.0,1.0,0.015,"""Jun11_06-49-13_seppel-liemarce…",0.248427
8.0,200.0,"""L""",20.0,1.0,0.0001,"""THREE_CLASS_STANDARD""",0.54,5.0,24.0,8.0,0.021,1.0,1.0,0.015,"""Jun12_07-38-49_seppel-liemarce…",0.233853
128.0,50.0,"""L""",30.0,0.0,0.0004,"""THREE_CLASS_STANDARD""",0.54,1.0,32.0,4.0,0.02,0.0,10.0,0.015,"""Jun05_14-40-09_seppel-liemarce…",0.206779


In [68]:
unique, identical = get_model_settings("cnn")
print(identical)
unique

shape: (1, 25)
┌────────────┬───────┬────────┬────────┬───┬─────────────┬───────────┬────────────────┬────────────┐
│ activation ┆ beats ┆ causal ┆ center ┆ … ┆ sample_rate ┆ scheduler ┆ splits         ┆ train_set  │
│ ---        ┆ ---   ┆ ---    ┆ ---    ┆   ┆ ---         ┆ ---       ┆ ---            ┆ ---        │
│ str        ┆ f64   ┆ f64    ┆ f64    ┆   ┆ f64         ┆ f64       ┆ str            ┆ str        │
╞════════════╪═══════╪════════╪════════╪═══╪═════════════╪═══════════╪════════════════╪════════════╡
│ SELU       ┆ 0.0   ┆ 1.0    ┆ 1.0    ┆ … ┆ 44100.0     ┆ 1.0       ┆ (0.85, 0.15,   ┆ a2md_train │
│            ┆       ┆        ┆        ┆   ┆             ┆           ┆ 0.0)           ┆            │
└────────────┴───────┴────────┴────────┴───┴─────────────┴───────────┴────────────────┴────────────┘


batch_size,dataset_version,epochs,flux,learning_rate,mapping,min_test_score,model_settings,num_channels,onset_cooldown,pad_annotations,test_batch_size,time_shift,dir_name,F-Score
f64,str,f64,f64,f64,str,f64,str,f64,f64,f64,f64,f64,str,f64
16.0,"""L""",20.0,1.0,0.0001,"""THREE_CLASS_STANDARD""",0.54,"""cnn_attention""",24.0,0.021,1.0,1.0,0.015,"""Jun13_10-57-29_seppel-liemarce…",0.492189
512.0,"""L""",30.0,1.0,0.0001,"""THREE_CLASS_STANDARD""",0.54,"""cnn""",16.0,0.02,0.0,10.0,0.015,"""Jun04_19-26-40_seppel-liemarce…",0.487235
16.0,"""L""",20.0,1.0,0.0001,"""THREE_CLASS_STANDARD""",0.54,"""cnn_attention""",32.0,0.021,1.0,1.0,0.015,"""Jun14_09-33-24_seppel-liemarce…",0.482672
512.0,"""L""",20.0,1.0,0.0001,"""THREE_CLASS_STANDARD""",0.54,"""cnn""",16.0,0.02,1.0,1.0,0.015,"""Jun10_09-14-54_seppel-liemarce…",0.462143
4.0,"""L""",20.0,0.0,0.0001,"""THREE_CLASS_STANDARD""",0.54,"""cnn_attention""",16.0,0.021,1.0,1.0,0.015,"""Jun11_13-54-21_seppel-liemarce…",0.458652
16.0,"""L""",20.0,1.0,0.0001,"""Three class standard""",0.48,"""cnn_attention""",24.0,0.021,1.0,1.0,0.02,"""Jun17_15-04-11_seppel-liemarce…",0.393078
512.0,"""L""",20.0,1.0,0.0001,"""THREE_CLASS_STANDARD""",0.54,"""cnn""",16.0,0.02,1.0,1.0,0.025,"""Jun09_16-46-27_seppel-liemarce…",0.267671
4.0,"""S""",20.0,0.0,0.0001,"""THREE_CLASS_STANDARD""",0.54,"""cnn_attention""",32.0,0.021,1.0,1.0,0.015,"""Jun11_06-49-13_seppel-liemarce…",0.248427
8.0,"""L""",20.0,1.0,0.0001,"""THREE_CLASS_STANDARD""",0.54,"""cnn_attention""",24.0,0.021,1.0,1.0,0.015,"""Jun12_07-38-49_seppel-liemarce…",0.233853
